<a href="https://colab.research.google.com/github/adikush1/Unemployment_Rate_Pred/blob/main/Unemployement_Rate_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below is the Dataset (top 5 entries) which we will be using for training the Logistic Regression model.

It takes 6 variables namely :

**-->Year**

**-->GDP per capita (current US$)**

**-->GDP growth (annual %)**

**-->Inflation,consumer prices (annual %)**

**-->Population growth (annual %)**

**-->Life expectancy at birth, total (years)**

**-->Unemployement rate**





In [38]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Unemployement_pred_dataset.xlsx',)
df.head()


,Year,GDP per capita (current US$),GDP growth (annual %),"Inflation, consumer prices (annual %)",Population growth (annual %),"Life expectancy at birth, total (years)",Unemployement rate
0,1991,303.0,1.06,13.87,2.12,58.15,6.737
1,1992,317.0,5.48,11.79,2.10,58.63,6.815
2,1993,301.0,4.75,6.33,2.07,59.12,6.798
3,1994,346.0,6.66,10.25,2.04,59.59,6.830
4,1995,374.0,7.57,10.22,2.01,60.06,7.014


*Importing Necessary Libraries and Defining the dataframes*

Now Importing the Logistic regression Model from 'sklearn' python libraray and importing other necessary libraries as well .

Defining dataframe and splitting the dataset into Train and Test data and converting the unemployment rate data into binary form using a threshold value of unemployment rate.

In [54]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Define the path to your dataset
dataset_path = '/content/drive/MyDrive/Unemployement_pred_dataset.xlsx'

# Load the dataset
df = pd.read_excel(dataset_path)

# Clean column names (remove leading and trailing whitespace)
df.columns = df.columns.str.strip()

# Drop the row corresponding to year 2020 containing the outliers due to the Covid19 pandemic
df = df.drop(29)

# Extract independent variables (features) and dependent variable (target)
X = df[['Year', 'GDP per capita (current US$)', 'GDP growth (annual %)', 'Inflation, consumer prices (annual %)', 'Population growth (annual %)', 'Life expectancy at birth, total (years)']]
Y = df['Unemployement rate']

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Apply feature scaling
scaler = StandardScaler()
X_imputed = scaler.fit_transform(X_imputed)


# Define the threshold to classify high and low unemployment




#X = pd.get_dummies(X)


#splitting the data
#'train' dataframe will be used to train the Data while 'test' will be used to test the model
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y, test_size=0.2, random_state=21)
print('Y_Test- Actual Values of Unemployment Rate during Test Data Years')
print(Y_test)
threshold = np.average(Y_test)

print('Threshold value of unemployment rate is:', threshold)

# Converting Unemployment Rate into Binary form:
# if (rate >=Threshold) : set rate to 0 (high unemployemnt)
# else if(rate < Threshold) : set rate to 1 (low unemployment)
df['Unemployment Category'] = df['Unemployement rate'].apply(lambda x: 0 if x >= threshold else 1)
#print(type('Unemployement rate'))
Y=df['Unemployment Category']
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y, test_size=0.2, random_state=21)
#print(type(Y_test))
#print(Y_test)
#converting y_test data into binary form depending on the threshold value
y_test_bin=np.zeros((len(Y_test),1))
for i in range(len(Y_test)):
  if Y_test.values[i]==1:
    y_test_bin[i]=1
  else:
    y_test_bin[i]=0


Y_Test- Actual Values of Unemployment Rate during Test Data Years
5     7.181
23    7.981
22    8.037
28    6.510
1     6.815
21    8.095
Name: Unemployement rate, dtype: float64
Threshold value of unemployment rate is: 7.436499999999999


*Applying Logistics Regression and calculation of the probability of low unemployement based on the threshold value set using the average value set by the user.*

*Also Calculating the Accuracy Score and F1 score.*

In [55]:
# Initialize the logistic regression model and solving through 'liblinear' method
logistic_regression = LogisticRegression(solver='liblinear', max_iter=100000)

# Fit the model to the training data
logistic_regression.fit(X_train,Y_train)

# Predict the probability of unemployment category for the testing data
y_pred_prob = logistic_regression.predict_proba(X_test)

# Convert X_test to a DataFrame for better visualization
X_test_df = pd.DataFrame(X_test, columns=X.columns)
y_pred_bin = np.zeros((len(y_pred_prob), 1))

# if predicted probability of low unemployment comes to be greater than 0.5 set it to 1 indicating
# high chances of Low unemployment rate else setting it to 1 indicating low chances of Low Unemployment Rate
for i in range(len(y_pred_prob)):
  if y_pred_prob[i][1]>=0.5:
    y_pred_bin[i][0]=1
  else:
     y_pred_bin[i][0]=0



# Add a new column to X_test_df for the predicted probabilities of the positive class
X_test_df['Predicted Probability (Low Unemployment)   '] = y_pred_prob[:, 1]
X_test_df['Predicted Prob in binary form'] = y_pred_bin
X_test_df['Actual Unemployment Rate     '] = df.loc[Y_test.index, 'Unemployement rate'].reset_index(drop=True)
X_test_df['Actual Unemployment in binary form'] = y_test_bin

print(X_test_df)




       Year  GDP per capita (current US$)  GDP growth (annual %)  \
0 -1.094156                     -0.932357               0.627057   
1  0.971733                      0.948205               0.554284   
2  0.856962                      0.749577               0.024084   
3  1.545592                      1.792376              -1.197455   
4 -1.553243                     -1.065310              -0.448937   
5  0.742190                      0.739966              -0.459333   

   Inflation, consumer prices (annual %)  Population growth (annual %)  \
0                               0.573060                      1.072521   
1                              -0.154952                     -0.924587   
2                               0.900823                     -0.758162   
3                              -1.081513                     -1.534815   
4                               1.458651                      1.433110   
5                               0.730639                     -0.674949   

   L

In [56]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Assuming you have true labels Y_true and predicted labels Y_pred

# Accuracy
accuracy = accuracy_score(Y_test, y_pred_bin)

# F1 score
f1 = f1_score(Y_test, y_pred_bin)

# Precision
precision = precision_score(Y_test, y_pred_bin)

# Recall
recall = recall_score(Y_test, y_pred_bin)

# Area Under the ROC Curve (AUC-ROC)
auc_roc = roc_auc_score(Y_test, y_pred_bin)

# Print the metrics
print("     Accuracy:", accuracy)
print("     F1 score:", f1)
print("     Precision:", precision)
print("     Recall:", recall)
print("     AUC-ROC:", auc_roc)


     Accuracy: 0.8333333333333334
     F1 score: 0.8
     Precision: 1.0
     Recall: 0.6666666666666666
     AUC-ROC: 0.8333333333333333
